In [34]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296186 sha256=820dbba9cd0129ec367bb9145d173efe7705362af22beafa0e73bf4621b094cb
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [35]:
import pandas as pd
import re
import fasttext

In [23]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/ecommerce_dataset.csv",names=['category','description'],header=None)
print(df.shape)
df.head()

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [24]:
df['category'].value_counts()

,count
category,
Household,19313
Books,11820
Electronics,10621
Clothing & Accessories,8671


In [25]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [26]:
df['category'].replace("Clothing & Accessories","Clothing_&_Accessories",inplace=True)
df.head()

<ipython-input-26-c0d74d49ffe7>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['category'].replace("Clothing & Accessories","Clothing_&_Accessories",inplace=True)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [27]:
df['category'] = "__label__" + df['category'].astype(str)
df.head()

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [28]:
df['category_description'] = df['category']+ " " + df['description']
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [29]:
df['category_description'][0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

In [30]:
def preprocess(text):
  text = re.sub(r'[^\w\s\']'," ",text)
  text = re.sub(r' +',' ',text)
  return text.strip().lower()

In [31]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [32]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,test_size=0.2)
train.shape, test.shape

((40339, 3), (10085, 3))

In [36]:
train.to_csv("/content/drive/MyDrive/Datasets/ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("/content/drive/MyDrive/Datasets/ecommerce.test", columns=["category_description"], index=False, header=False)

In [37]:
model = fasttext.train_supervised(input="/content/drive/MyDrive/Datasets/ecommerce.train")

In [38]:
model.test("/content/drive/MyDrive/Datasets/ecommerce.test")

(10085, 0.9726326227069906, 0.9726326227069906)

In [41]:
model.predict("this mobile works with battery of 5000 mah")

(('__label__electronics',), array([0.5994001]))

In [42]:
model.predict("this novel has 400 pages with description of different planets")

(('__label__books',), array([0.92475933]))

In [45]:
model.get_nearest_neighbors("recharge")

[(0.8073331713676453, 'powder'),
 (0.8059592247009277, 'possession'),
 (0.8059485554695129, 'leoie'),
 (0.8058390617370605, 'technically'),
 (0.8058072924613953, 'proceed'),
 (0.8052137494087219, 'vari'),
 (0.8049184679985046, 'gizmo'),
 (0.8043766617774963, 'upscale'),
 (0.804349422454834, 'damask'),
 (0.8042137622833252, 'easily')]